In [94]:
SAMPLE_TEXT = """
1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581
"""

SAMPLE_TEXT_2 = """
11637517422274862853338597396444961841755517295286
13813736722492484783351359589446246169155735727126
21365113283247622439435873354154698446526571955763
36949315694715142671582625378269373648937148475914
74634171118574528222968563933317967414442817852555
13191281372421239248353234135946434524615754563572
13599124212461123532357223464346833457545794456865
31254216394236532741534764385264587549637569865174
12931385212314249632342535174345364628545647573965
23119445813422155692453326671356443778246755488935
22748628533385973964449618417555172952866628316397
24924847833513595894462461691557357271266846838237
32476224394358733541546984465265719557637682166874
47151426715826253782693736489371484759148259586125
85745282229685639333179674144428178525553928963666
24212392483532341359464345246157545635726865674683
24611235323572234643468334575457944568656815567976
42365327415347643852645875496375698651748671976285
23142496323425351743453646285456475739656758684176
34221556924533266713564437782467554889357866599146
33859739644496184175551729528666283163977739427418
35135958944624616915573572712668468382377957949348
43587335415469844652657195576376821668748793277985
58262537826937364893714847591482595861259361697236
96856393331796741444281785255539289636664139174777
35323413594643452461575456357268656746837976785794
35722346434683345754579445686568155679767926678187
53476438526458754963756986517486719762859782187396
34253517434536462854564757396567586841767869795287
45332667135644377824675548893578665991468977611257
44961841755517295286662831639777394274188841538529
46246169155735727126684683823779579493488168151459
54698446526571955763768216687487932779859814388196
69373648937148475914825958612593616972361472718347
17967414442817852555392896366641391747775241285888
46434524615754563572686567468379767857948187896815
46833457545794456865681556797679266781878137789298
64587549637569865174867197628597821873961893298417
45364628545647573965675868417678697952878971816398
56443778246755488935786659914689776112579188722368
55172952866628316397773942741888415385299952649631
57357271266846838237795794934881681514599279262561
65719557637682166874879327798598143881961925499217
71484759148259586125936169723614727183472583829458
28178525553928963666413917477752412858886352396999
57545635726865674683797678579481878968159298917926
57944568656815567976792667818781377892989248891319
75698651748671976285978218739618932984172914319528
56475739656758684176786979528789718163989182927419
67554889357866599146897761125791887223681299833479
"""

In [2]:
from dataclasses import dataclass
from typing import Dict, Set, Optional, List

In [3]:
def tokenize_line(line):
    return [int(i) for i in list(line)]

def parse_text(raw_text):
    return [tokenize_line(l) for l in raw_text.split("\n") if l]

def read_input():
    with open("input.txt", "rt") as f:
        return f.read()

In [144]:
# Use Dijkstra's algorithm since it will definitely work.

@dataclass(frozen=True)
class Position:
    x: int
    y: int

@dataclass
class Cost:
    parent: Position
    cost: int

    def update(self, new_parent: Position, new_cost: int) -> bool:
        if new_cost < self.cost:
            self.parent = new_parent
            self.cost = new_cost
            return True
        return False

def find_cheapest_node(costs: Dict[Position, Cost], visited: Set[Position]) -> Optional[Position]:
    non_visited_costs = (i for i in costs.items() if i[0] not in visited)
    sorted_costs = sorted(non_visited_costs, key=lambda x: x[1].cost)
    if not sorted_costs:
        return None
    return sorted_costs[0][0]

def get_route(start: Position, end: Position, costs: Dict[Position, Cost]) -> List[Position]:
    result = [end]
    while True:
        parent = costs[result[-1]].parent
        result.append(parent)
        if parent == start:
            break
    result.reverse()
    return result

def get_adjacent_positions(from_position: Position, weights: Dict[Position, int], visited: Set[Position]) -> List[Position]:
    possible_positions = [
        Position(from_position.x - 1, from_position.y),
        Position(from_position.x + 1, from_position.y),
        Position(from_position.x, from_position.y - 1),
        Position(from_position.x, from_position.y + 1),
    ]
    return [p for p in possible_positions if p in weights and p not in visited]

def find_cheapest_path(weights: Dict[Position, int], start: Position, end: Position):
    costs: Dict[Position, Cost] = dict()
    visited: Set[Position] = set()

    if start == end:
        return 0, []

    visited.add(start)
    for neighbour in get_adjacent_positions(start, weights, visited):
        costs[neighbour] = Cost(start, weights[neighbour])

    node = find_cheapest_node(costs, visited)
    i = 0
    while node is not None:
        i += 1
        if i == 10000:
            pct_visited = len(visited) / ((end.x + 1) * (end.y + 1))
            print(pct_visited)
            i = 0
        running_total = costs[node].cost
        for neighbour in get_adjacent_positions(node, weights, visited):
            new_cost = running_total + weights[neighbour]
            if neighbour not in costs:
                costs[neighbour] = Cost(node, new_cost)
            else:
                costs[neighbour].update(node, new_cost)
        visited.add(node)
        node = find_cheapest_node(costs, visited)
    if end not in costs:
        raise ValueError(f"No route to {end}")
    return costs[end], get_route(start, end, costs)

def make_weights(lines) -> Dict[Position, int]:
    result = {}
    for y in range(len(lines)):
        for x in range(len(lines[0])):
            result[Position(x, y)] = lines[y][x]
    return result

def find_end(weights: Dict[Position, int]) -> Position:
    max_x = 0
    max_y = 0
    for k in weights.keys():
        max_x = max(max_x, k.x)
        max_y = max(max_y, k.y)
    return Position(max_x, max_y)

def print_weights(weights: Dict[Position, int]):
    end = find_end(weights)
    result = ""
    for y in range(end.y + 1):
        for x in range(end.x + 1):
            result += str(weights[Position(x, y)])
        result += '\n'
    print(result)

In [137]:
weights = make_weights(parse_text(SAMPLE_TEXT))
cost, route = find_cheapest_path(weights, Position(0, 0), find_end(weights))
cost.cost

% visited 1.0


40

In [138]:
weights = make_weights(parse_text(read_input()))
cost, route = find_cheapest_path(weights, Position(0, 0), find_end(weights))
cost.cost

% visited 1.0


458

In [104]:
print_weights(make_weights(parse_text(SAMPLE_TEXT)))

1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581



In [120]:
def multiply_weights(original_weights: Dict[Position, int], times):
    end = find_end(original_weights)
    new_max_x = (end.x + 1) * times - 1
    new_max_y = (end.y + 1) * times - 1
    result = {}
    for x in range(new_max_x + 1):
        for y in range(new_max_y + 1):
            orig_x = x % (end.x + 1)
            orig_y = y % (end.y + 1)
            multiplier = (x // (end.x + 1)) + (y // (end.y + 1))
            original_value = original_weights[Position(orig_x, orig_y)]
            new_value = original_value + multiplier
            if new_value > 9:
                new_value -= 9
            result[Position(x, y)] = new_value
    return result

In [121]:
multiply_weights(make_weights(parse_text(SAMPLE_TEXT)), 5) == make_weights(parse_text(SAMPLE_TEXT_2))

True

In [140]:
new_weights = multiply_weights(make_weights(parse_text(SAMPLE_TEXT)), 5)
cost, route = find_cheapest_path(new_weights, Position(0, 0), find_end(new_weights))
cost.cost

315

In [145]:
# Need a better solution for the big map. The approach above takes way too long
new_weights = multiply_weights(make_weights(parse_text(read_input())), 5)
cost, route = find_cheapest_path(new_weights, Position(0, 0), find_end(new_weights))
cost.cost

0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96


2800